In [1]:
import numpy as np
import pandas as pd
import requests
import json
#from scipy import stats as st

In [2]:
import requests
import json


class MakeApiCall:

    def get_data(self, api):
        response = requests.get(f"{api}")
        if response.status_code == 200:
            print("sucessfully fetched the data")
            self.formatted_print(response.json())
        else:
            print(
                f"Hello person, there's a {response.status_code} error with your request")

    def get_user_data(self, api, parameters):
        response = requests.get(f"{api}", headers=parameters)
        if response.status_code == 200:
            print("sucessfully fetched the data with parameters provided")
            self.formatted_print(response.json())
        else:
            print(
                f"Hello person, there's a {response.status_code} error with your request")

    def formatted_print(self, obj):
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)

    def __init__(self, api):
         self.get_data(api)

        #parameters = {
            #'Authorization': 'access_token myToken'
        #}
        #self.get_user_data(api, parameters)


Separate SF into tiles w/ lat and long look ups

In [3]:
# SF is about ~15 km x 15 km
# The paper used 2 km x 2 km tiles
# Let's use 100 m x 100 m tile 

import geopy.distance
import math

#longest vertical distance
coords_1 = (37.811262, -122.477010)
coords_2 = (37.708251, -122.477010)

vertical_dist = geopy.distance.geodesic(coords_1, coords_2).m
v_count = math.ceil(vertical_dist / 100)

#longest horizontal distance 
coords_1 = (37.778946, -122.514446)
coords_2 = (37.778946, -122.357110)

horizontal_dist = geopy.distance.geodesic(coords_1, coords_2).m
h_count = math.ceil(horizontal_dist / 100)

In [4]:
# https://code.activestate.com/recipes/577713-midpoint-of-two-gps-points/ 

import math

def midpoint(x1, y1, x2, y2):
#Input values as degrees

#Convert to radians
    lat1 = math.radians(x1)
    lon1 = math.radians(x2)
    lat2 = math.radians(y1)
    lon2 = math.radians(y2)


    bx = math.cos(lat2) * math.cos(lon2 - lon1)
    by = math.cos(lat2) * math.sin(lon2 - lon1)
    lat3 = math.atan2(math.sin(lat1) + math.sin(lat2), \
           math.sqrt((math.cos(lat1) + bx) * (math.cos(lat1) \
           + bx) + by**2))
    lon3 = lon1 + math.atan2(by, math.cos(lat1) + bx)

    return [round(math.degrees(lat3), 6), round(math.degrees(lon3), 6)]

In [5]:
# If midpoint of tile is in the ocean, exclude
# https://gis.stackexchange.com/questions/235133/checking-if-a-geocoordinate-point-is-land-or-ocean

from mpl_toolkits.basemap import Basemap
bm = Basemap()   # default: projection='cyl'
print (Basemap().is_land(99.675, 13.104))  #True
print (Basemap().is_land(100.539, 13.104))  #False
lon, lat = (11.61168822665539, 57.86868795294363) # test coords
xpt, ypt = bm( 11.61168822665539, 57.86868795294363 ) # convert to projection map
value = bm.is_land(xpt, ypt) # test is_land
value

True
False


False

In [ ]:
bm = Basemap(projection='cyl', resolution='f')   # default: projection='cyl'

In [6]:
# Make a dataframe with:
# Tile #
# Upper left coordinate
# Upper right
# Lower left
# lower right 
# midpoint 

long_interval = np.linspace(-122.357110, -122.514446, h_count)

lat_interval = np.linspace(37.708251, 37.811262, v_count)

tiles = pd.DataFrame(columns = ['Tile_ID', 'Lat1', 'Lat2', 'Long1', 'Long2', 'Mid_lat', 'Mid_long'])

count1 = 0

bm = Basemap(projection='cyl', resolution='f') 

for lai in range(len(lat_interval) - 1): 
    lat1 = lat_interval[lai]
    lat2 = lat_interval[lai + 1]
    for loi in range(len(long_interval) - 1):
        long1 = long_interval[loi]
        long2 = long_interval[loi + 1]
        mid = midpoint(lat1, lat2, long1, long2)
        ypt,xpt = bm(mid[1], mid[0]) #converting to projection coordinates, enter parameters as long, lat 
        if bm.is_land(ypt,xpt) == False:
            #print(mid[0], mid[1])
            continue
        #print("added", lai, loi)
        t = pd.DataFrame([[count1, lat1, lat2, long1, long2, mid[0], mid[1]]], columns = ['Tile_ID', 'Lat1', 'Lat2', 'Long1', 'Long2', 'Mid_lat', 'Mid_long'])
        tiles = pd.concat([tiles, t], ignore_index = True)
        count1 += 1


In [8]:
tiles.to_csv('.\Data\Tiles.csv')

In [9]:
tiles_2 = pd.read_csv(r'.\Data\Tiles.csv')

In [10]:
tiles_2.head(5)

,Unnamed: 0,Tile_ID,Lat1,Lat2,Long1,Long2,Mid_lat,Mid_long
0,0,0,37.708251,37.709155,-122.391313,-122.392454,37.708703,-122.391884
1,1,1,37.708251,37.709155,-122.392454,-122.393594,37.708703,-122.393024
2,2,2,37.708251,37.709155,-122.393594,-122.394734,37.708703,-122.394164
3,3,3,37.708251,37.709155,-122.394734,-122.395874,37.708703,-122.395304
4,4,4,37.708251,37.709155,-122.395874,-122.397014,37.708703,-122.396444


In [11]:
len(tiles_2)

12227